In [ ]:
!pip install torchcam

In [ ]:
!pip install saliency

In [ ]:
!pip install captum

In [ ]:
import torch
from torchvision import models, transforms
from captum.attr import GuidedBackprop
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
from captum.attr import GuidedGradCam
import os
from matplotlib import pylab as P
import PIL.Image
import saliency.core as saliency

In [ ]:

##### Specify Device

device = torch.device("cpu")

if torch.cuda.is_available():
   print("Training on GPU")
   device = torch.device("cuda:0")


#Preliminary

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [ ]:

# Boilerplate methods.
def ShowImage(im, title='', ax=None):
    if ax is None:
        P.figure()
    P.axis('off')
    P.imshow(im)
    P.title(title)

def ShowGrayscaleImage(im, title='', ax=None):
    if ax is None:
        P.figure()
    P.axis('off')
    P.imshow(im, cmap=P.cm.gray, vmin=0, vmax=1)
    P.title(title)

def ShowHeatMap(im, title, ax=None):
    if ax is None:
        P.figure()
    P.axis('off')
    P.imshow(im, cmap='inferno')
    P.title(title)

def LoadImage(file_path):
    im = PIL.Image.open(file_path).convert('RGB')
    im = im.resize((224, 224))
    im = np.asarray(im)
    return im

transformer = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
def PreprocessImagesNOBATCH(images):
    # assumes input is 4-D, with range [0,255]
    images = np.array(images)
    images = images/255
    #print(images.shape)
    images = images.reshape(1,224,224,3)
    images = np.transpose(images, (0,3,1,2))
    images = torch.tensor(images, dtype=torch.float32)
    images = images.to(device)
    images = transformer.forward(images)
    return images.requires_grad_(True)

def PreprocessImages(images):
    # assumes input is 4-D, with range [0,255]
    images = np.array(images)
    images = images/255
    #print(images.shape)
    #images = images.reshape(1,299,299,3)
    images = np.transpose(images, (0,3,1,2))
    images = torch.tensor(images, dtype=torch.float32)
    images = transformer.forward(images)
    return images.requires_grad_(True)



In [ ]:

# Load the pre-trained model
model = torch.load('/content/drive/MyDrive/Saliency Map Research 2023/Models/pneumonia_dense.pt')
model=model.to(device)
model.eval()

In [ ]:
# Register hooks for Grad-CAM, which uses the last convolution layer
conv_layer = model.features.denseblock4.denselayer16.conv2
conv_layer_outputs = {}
def conv_layer_forward(m, i, o):
    # move the RGB dimension to the last dimension
    conv_layer_outputs[saliency.base.CONVOLUTION_LAYER_VALUES] = torch.movedim(o.cpu(), 1, 3).detach().numpy()
def conv_layer_backward(m, i, o):
    # move the RGB dimension to the last dimension
    conv_layer_outputs[saliency.base.CONVOLUTION_OUTPUT_GRADIENTS] = torch.movedim(o[0].cpu(), 1, 3).detach().numpy()

conv_layer.register_forward_hook(conv_layer_forward)
conv_layer.register_full_backward_hook(conv_layer_backward)

In [ ]:
class_idx_str = 'class_idx_str'
def call_model_function(images, call_model_args=None, expected_keys=None):
    images = PreprocessImages(images)
    images = images.to(device)
    target_class_idx =  call_model_args[class_idx_str]
    output = model(images)
    m = torch.nn.Softmax(dim=1)
    output = m(output)
    if saliency.base.INPUT_OUTPUT_GRADIENTS in expected_keys:
        outputs = output[:,target_class_idx]
        grads = torch.autograd.grad(outputs, images, grad_outputs=torch.ones_like(outputs))
        grads = torch.movedim(grads[0].cpu(), 1, 3)
        gradients = grads.detach().numpy()
        return {saliency.base.INPUT_OUTPUT_GRADIENTS: gradients}
    else:
        one_hot = torch.zeros_like(output)
        one_hot[:,target_class_idx] = 1
        model.zero_grad()
        output.backward(gradient=one_hot, retain_graph=True)
        return conv_layer_outputs

In [ ]:
#Count files in directory

directory_path = '/content/drive/MyDrive/Saliency Map Research 2023/SaliencyMaps/Pneumonia Resnet (Tensor)/GradCAM'
file_count = 0

# Iterate over all the files and directories in the given directory
for root, dirs, files in os.walk(directory_path):
    file_count += len(files)

print("GradCAM:", file_count)

directory_path = '/content/drive/MyDrive/Saliency Map Research 2023/SaliencyMaps/Pneumonia Resnet (Tensor)/Guided Backprop'
file_count = 0

# Iterate over all the files and directories in the given directory
for root, dirs, files in os.walk(directory_path):
    file_count += len(files)

print("Guided Backprop:", file_count)

directory_path = '/content/drive/MyDrive/Saliency Map Research 2023/SaliencyMaps/Pneumonia Resnet (Tensor)/Guided GradCAM'
file_count = 0

# Iterate over all the files and directories in the given directory
for root, dirs, files in os.walk(directory_path):
    file_count += len(files)

print("Guided GradCAM", file_count)

directory_path = '/content/drive/MyDrive/Saliency Map Research 2023/SaliencyMaps/Pneumonia Resnet (Tensor)/GradCAM++'
file_count = 0

# Iterate over all the files and directories in the given directory
for root, dirs, files in os.walk(directory_path):
    file_count += len(files)

print("GradCAM++:", file_count)

directory_path = '/content/drive/MyDrive/Saliency Map Research 2023/SaliencyMaps/Pneumonia Resnet (Tensor)/ScoreCAM'
file_count = 0

# Iterate over all the files and directories in the given directory
for root, dirs, files in os.walk(directory_path):
    file_count += len(files)

print("ScoreCAM:", file_count)

In [ ]:

def make_labels(file_path):
  labels = pd.read_csv(file_path)
  labels = labels.drop(['x','y','width','height'], axis=1)
  labels = labels.drop_duplicates("patientId")
  return labels

labels = make_labels("/content/drive/MyDrive/Saliency Map Research 2023/Pneumonia Dataset/Pneumonia_Labels_Final_2.csv")
patientId = labels['patientId'].tolist()
#lim_patientId = patientId[13390:]

root = '/content/drive/MyDrive/Saliency Map Research 2023/Pneumonia Dataset/stage_2_train_images_png'


#Script


In [ ]:
from torchcam.methods import GradCAM, ScoreCAM, GradCAMpp
from torchcam.utils import overlay_mask
from torchvision.transforms.functional import normalize, resize, to_pil_image
from matplotlib import cm
import time

model = torch.load('/content/drive/MyDrive/Saliency Map Research 2023/Models/pneumonia_dense.pt').eval()
model = model.to(device)

start_time = time.time()

counter = 0

model.requires_grad_(True)

for label in patientId:

  counter += 1  # Increment the counter

  if counter % 10 == 0:  # Display the count for every 10th image
       print(f"Processing image {counter}/{len(patientId)}")

  path = os.path.join(root, label + '.dcm.jpg.png')

  im = LoadImage(path)
  im_tensor = PreprocessImagesNOBATCH(im)
  predictions = model(im_tensor)
  predictions = predictions.detach().cpu().numpy()
  prediction_class = np.argmax(predictions[0])
  input_image = im_tensor

  prediction_class = torch.tensor(prediction_class)

  if torch.cuda.is_available():
      input_image = input_image.cuda()  # Move input tensor to GPU

  ######Guided Back Propagation######

  # Generate saliency map using Guided Backpropagation
  guided_backprop = GuidedBackprop(model)
  guided_backprop_map = guided_backprop.attribute(input_image, target=prediction_class)  # Specify the target class index

  saliency_map_gbp = np.transpose(guided_backprop_map.squeeze().cpu().numpy(), (1, 2, 0))

  ######GradCAM######

  cam_extractor = GradCAM(model, target_layer=model.layer4[-1].conv3)

  out = model(input_image)
  cam = cam_extractor(out.squeeze(0).argmax().item(), out)

  cam = torch.tensor(cam[0])

  img = to_pil_image(im)

  cmap = cm.get_cmap("plasma")

  mask = to_pil_image(cam.squeeze(0), mode='F')

  overlay = mask.resize(img.size, resample=Image.BICUBIC)
  overlay = cmap(np.asarray(overlay) ** 2)
  overlay = overlay[:,:,:3]

  saliency_map_gc = overlay

  ######Guided GradCAM######

  saliency_map_ggc = saliency_map_gbp * saliency_map_gc

  cam_extractor.remove_hooks()

  ######GradCAM++######

  cam_extractor = GradCAMpp(model, target_layer=model.layer4[-1].conv3)

  out = model(input_image)
  cam = cam_extractor(out.squeeze(0).argmax().item(), out)

  cam = torch.tensor(cam[0])

  mask = to_pil_image(cam.squeeze(0), mode='F')

  overlay = mask.resize(img.size, resample=Image.BICUBIC)
  overlay = cmap(np.asarray(overlay) ** 2)
  overlay = overlay[:,:,:3]

  saliency_map_gcpp = overlay

  cam_extractor.remove_hooks()

  ######ScoreCAM######

  cam_extractor = ScoreCAM(model, target_layer=model.layer4[-1].conv3)

  out = model(input_image)
  cam = cam_extractor(out.squeeze(0).argmax().item(), out)

  cam = torch.tensor(cam[0])

  mask = to_pil_image(cam.squeeze(0), mode='F')

  overlay = mask.resize(img.size, resample=Image.BICUBIC)
  overlay = cmap(np.asarray(overlay) ** 2)
  overlay = overlay[:,:,:3]

  saliency_map_sc = overlay

  cam_extractor.remove_hooks()

  #Save the arrays
  torch.save(saliency_map_gbp, os.path.join('/content/drive/MyDrive/Saliency Map Research 2023/SaliencyMaps/Pneumonia Resnet (Tensor)/Guided Backprop', label + '_saliency_map.pt'))
  torch.save(saliency_map_gc, os.path.join('/content/drive/MyDrive/Saliency Map Research 2023/SaliencyMaps/Pneumonia Resnet (Tensor)/GradCAM', label + '_saliency_map.pt'))
  torch.save(saliency_map_ggc, os.path.join('/content/drive/MyDrive/Saliency Map Research 2023/SaliencyMaps/Pneumonia Resnet (Tensor)/Guided GradCAM', label + '_saliency_map.pt'))
  torch.save(saliency_map_gcpp, os.path.join('/content/drive/MyDrive/Saliency Map Research 2023/SaliencyMaps/Pneumonia Resnet (Tensor)/GradCAM++', label + '_saliency_map.pt'))
  torch.save(saliency_map_sc, os.path.join('/content/drive/MyDrive/Saliency Map Research 2023/SaliencyMaps/Pneumonia Resnet (Tensor)/ScoreCAM', label + '_saliency_map.pt'))

end_time = time.time()
execution_time = end_time - start_time

print("Execution time:", execution_time, "seconds")


#Visualize

In [ ]:
gbp_mask_grayscale = saliency.VisualizeImageGrayscale(saliency_map_gbp)

# Set up matplot lib figures.
ROWS = 1
COLS = 1
UPSCALE_FACTOR = 10
P.figure(figsize=(ROWS * UPSCALE_FACTOR, COLS * UPSCALE_FACTOR))

# Render the saliency masks.
ShowHeatMap(gbp_mask_grayscale, title='Guided Backpropagation', ax=P.subplot(ROWS, COLS, 1))

In [ ]:
gbp_mask_grayscale = saliency.VisualizeImageGrayscale(saliency_map_gc)

# Set up matplot lib figures.
ROWS = 1
COLS = 1
UPSCALE_FACTOR = 10
P.figure(figsize=(ROWS * UPSCALE_FACTOR, COLS * UPSCALE_FACTOR))

# Render the saliency masks.
ShowHeatMap(gbp_mask_grayscale, title='Grad CAM', ax=P.subplot(ROWS, COLS, 1))

In [ ]:
gbp_mask_grayscale = saliency.VisualizeImageGrayscale(saliency_map_ggc)

# Set up matplot lib figures.
ROWS = 1
COLS = 1
UPSCALE_FACTOR = 10
P.figure(figsize=(ROWS * UPSCALE_FACTOR, COLS * UPSCALE_FACTOR))

# Render the saliency masks.
ShowHeatMap(gbp_mask_grayscale, title='Guided Grad CAM', ax=P.subplot(ROWS, COLS, 1))

In [ ]:
gbp_mask_grayscale = saliency.VisualizeImageGrayscale(saliency_map_gcpp)

# Set up matplot lib figures.
ROWS = 1
COLS = 1
UPSCALE_FACTOR = 10
P.figure(figsize=(ROWS * UPSCALE_FACTOR, COLS * UPSCALE_FACTOR))

# Render the saliency masks.
ShowHeatMap(gbp_mask_grayscale, title='Grad CAM++', ax=P.subplot(ROWS, COLS, 1))

In [ ]:
gbp_mask_grayscale = saliency.VisualizeImageGrayscale(saliency_map_sc)

# Set up matplot lib figures.
ROWS = 1
COLS = 1
UPSCALE_FACTOR = 10
P.figure(figsize=(ROWS * UPSCALE_FACTOR, COLS * UPSCALE_FACTOR))

# Render the saliency masks.
ShowHeatMap(gbp_mask_grayscale, title='Score CAM', ax=P.subplot(ROWS, COLS, 1))